In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
df_fc = pd.read_csv("data_sets/fragility_multiple_curves_piping.csv")
df_fc1 = df_fc[(df_fc["section_id"] == 1) & (df_fc["scenario_id"] == 1)]
df_fc1

In [ ]:
data = np.array(
    [
        [10, 2.202],
        [30, 2.349],
        [100, 2.484],
        [300, 2.594],
        [1000, 2.708],
        [3000, 2.812],
        [10000, 2.939],
        [30000, 3.090],
        [100000, 3.303],
    ]
)
df_s = pd.DataFrame(data, columns=["failure_probability", "waterlevel"])
df_s["waterlevel"] += 2
df_s["failure_probability"] = 1 / df_s["failure_probability"]
df_s

In [ ]:
fig, ax = plt.subplots()
df_fc1.plot(x="hydraulicload", y="failure_probability", ax=ax, color="C1")
ax2 = ax.twinx()
df_s.plot(x="waterlevel", ax=ax2, logy=True)

In [ ]:
def interpolate_1d(x, xp, fp):
    """
    Interpolate an array along the given axis.
    Similar to np.interp, but with extrapolation outside range.

    Parameters
    ----------
    x : np.array
        Array with positions to interpolate at
    xp : np.array
        Array with positions of known values
    fp : np.array
        Array with values as known positions to interpolate between

    Returns
    -------
    np.array
        interpolated array
    """
    # Determine lower bounds
    intidx = np.minimum(np.maximum(0, np.searchsorted(xp, x) - 1), len(xp) - 2)
    # Determine interpolation fractions
    fracs = (x - xp[intidx]) / (xp[intidx + 1] - xp[intidx])
    # Interpolate (1-frac) * f_low + frac * f_up
    f = (1 - fracs) * fp[intidx] + fp[intidx + 1] * fracs

    return f

In [ ]:
def log_interpolate_1d(x, xp, fp, ll=1e-20):
    """Similar to interpolate_1d, but interpolates in log-space"""
    if ll > 0:
        fp[fp < ll] = ll
    return np.exp(interpolate_1d(x, xp, np.log(fp)))

In [ ]:
def _int_midpoint(wl_grid, pf_grid, hfreq_fq_grid):
    # Voor de bepaling van de integraal passen we de midpoint rule toe,
    # dichtheid van frequentie is midden van verschil tussen twee ov.freq. waarden
    # N.B. De stapgrootte van de waterstand dh valt weg in de formule.
    # (2*dh/2) * cond_faalkans(h) * (ov.freq.(h-dh)-ov.freq.(h+dh)) / (2*dh)
    # prob_fail_grid[i] = pf_grid[i] * (hfreq_fq_grid[i-1] - hfreq_fq_grid[i+1])/2
    prob_fail_grid = np.zeros(len(wl_grid))
    prob_fail_grid[1:-1] = (hfreq_fq_grid[0:-2] - hfreq_fq_grid[2:]) / 2
    prob_fail_grid[1:-1] = pf_grid[1:-1] * prob_fail_grid[1:-1]

    return prob_fail_grid

In [ ]:
wl_step = 0.2
wl_ll = 0
wl_ul = 10
wl_grid = np.linspace(wl_ll, wl_ul, int((wl_ul - wl_ll) / wl_step + 1))

In [ ]:
fc_interp = log_interpolate_1d(
    wl_grid,
    df_fc1["hydraulicload"].to_numpy(),
    df_fc1["failure_probability"].to_numpy(),
    ll=0,
)
stat_interp = log_interpolate_1d(
    wl_grid,
    df_s["waterlevel"].to_numpy(),
    df_s["failure_probability"].to_numpy(),
    ll=0,
)
stat_interp[stat_interp > 360] = 360
# stat_interp

In [ ]:
fig, ax = plt.subplots()

ax.plot(
    df_fc1["hydraulicload"].to_numpy(),
    df_fc1["failure_probability"].to_numpy(),
    color="C1",
    lw=4,
    ls="--",
)
ax.plot(wl_grid, fc_interp, color="C0", label="Interpolated fragility curve")

ax2 = ax.twinx()
df_s.plot(x="waterlevel", ax=ax2, color="C2", lw=4, ls="--")
ax2.semilogy(wl_grid, stat_interp, color="C2", label="Interpolated stat")
# ax2.set_ylim([0, 1e6])
ax2.legend().remove()

handles, labels = ax.get_legend_handles_labels()
handles_, labels_ = ax2.get_legend_handles_labels()
ax.legend(handles + handles_, labels + labels_);

In [ ]:
fp_grid = fc_interp.copy()
update_level = 4.2
trust_factor = 1

sel_update = wl_grid < update_level
wl_steps = np.diff(wl_grid[sel_update])
wl_steps = np.hstack([wl_steps[0], wl_steps])
F_update = trust_factor * (fp_grid[sel_update] * wl_steps).sum()

fp_grid[sel_update] = (1 - trust_factor) * fp_grid[sel_update]
fp_grid[~sel_update] = (fp_grid[~sel_update] - F_update) / (1 - F_update)


faalkansbijdrage = _int_midpoint(wl_grid, fc_interp, stat_interp)
faalkansbijdrage_upd = _int_midpoint(wl_grid, fp_grid, stat_interp)

fig, ax = plt.subplots()
ax.bar(wl_grid, faalkansbijdrage, width=0.1)
ax.bar(wl_grid, faalkansbijdrage_upd, width=0.1, alpha=0.95)
faalkansbijdrage.sum(), faalkansbijdrage_upd.sum()